In [7]:
"""
ChatPDF 简化版本
使用 LangChain 标准组件实现 PDF 文档问答系统
"""
import os
from langchain.retrievers import MultiQueryRetriever
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.llms import Tongyi

# 配置参数
EMBEDDING_MODEL = "text-embedding-v1"
LLM_MODEL = "qwen-turbo"
DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')

if not DASHSCOPE_API_KEY:
    raise ValueError("请设置 DASHSCOPE_API_KEY 环境变量")

# 创建LLM
llm = Tongyi(
    model_name=LLM_MODEL,
    temperature=0.7,
    dashscope_api_key=DASHSCOPE_API_KEY
)
embeddings = DashScopeEmbeddings(
    model=EMBEDDING_MODEL,
    dashscope_api_key=DASHSCOPE_API_KEY
)

 

In [25]:
# 加载向量数据库，添加allow_dangerous_deserialization=True参数以允许反序列化
vectorstore = FAISS.load_local("./faiss-1", embeddings, allow_dangerous_deserialization=True)
# 创建MultiQueryRetriever
#当使用 MultiQueryRetriever 时，即使设置了 search_kwargs={"k": 3}，最终返回的文档数量仍可能超过 3 个，
# 用大模型（LLM）将原始查询 扩展成多个相似问题（默认生成 3-5 个变体问题）
# 对每个变体问题 独立执行检索，每个都返回 k 个文档）
# 合并所有结果后去重
retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever( search_kwargs={"k": 2}),
    llm=llm
)



In [14]:
vectorstore

In [15]:
retriever

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['FAISS', 'DashScopeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000237D989FD40>, search_kwargs={'k': 5, 'score_threshold': 0.65}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}')
| Tongyi(client=<class 'dashscope.aigc.generation.Generation'>, model_kwargs={})
| LineListOutputParser())

In [26]:
# 示例查询
query = "客户经理的考核标准是什么？"
# 执行查询
#results = retriever.get_relevant_documents(query) #  The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46
results = retriever.invoke(query)
# 打印结果
print(f"查询: {query}")
print(f"找到 {len(results)} 个相关文档:")
for i, doc in enumerate(results):
    print(f"\n文档 {i+1}:")
    print(doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content)

查询: 客户经理的考核标准是什么？
找到 2 个相关文档:

文档 1:
-5 第五章  工作质量考核标准  
第九条   工作质量考核实行扣分制。工作质量指个金客户经理在
从事所有个人业务时出现投诉、差错及风险。该项考核最多扣 50分，
如发生重大差错事故，按分行有关制度处理。  
（一）服务质量考核：   
1、工作责任心不强，缺乏配合协作精神；扣 5分 
2、客户服务效率低，态度生硬或不及时为客户提供维护服务，
有客户投诉的 ,每投诉一次扣 2分 
3、不服从支行...

文档 2:
调整浮动。   
第十三条   特别聘任：  
（一）经分行同意录用从其他单位调入的个金客户经理，由用人
单位按 D类人员进行考核， 薪资待遇按其业绩享受行内正式行员工同
等待遇。待正式转正后按第十一条规定申报技术职务。  
（二）对为我行业务创新、工作业绩等方面做出重大贡献的市场
人员经支行推荐、分行行长 批准可越级聘任。  
第十四条   对于创利业绩较高，而暂未入围技术职务系列，或所
评聘技...
